In [ ]:
# importing the requests library 
# Referenced from: https://blog.rapid7.com/2018/10/16/how-to-conduct-dns-reconnaissance-for-02-using-rapid7-open-data-and-aws/

import requests
import pandas as pd
import io
import time

# ESTABLISH VARIABLES
# Special thanks to Evan Perotti for the awesome walkthrough to create the APIs and supporting AWS infrastructure!
# To create the AWS API endpoints, you will need to follow the blog at http://securityriskadvisors.com/blog/creating-a-project-sonar-fdns-api-with-aws/

# API Endpoints - You will need to input your specific URL to the API created using the above steps.
runQueryURL = "https://<api_name>.execute-api.us-east-1.amazonaws.com/production/{search-r7fqdns+}"
getResultsURL = "https://<api_name>.execute-api.us-east-1.amazonaws.com/production/{results-r7fqdns+}"

# Domain to search  - Enter your domain that you wish to query.
location = "derbycon.com"

# defining a params dict for the parameters to be sent to the API 
searchDomain = {'domain':location} 
  
# sending get request and saving the response as response object 
rSearch = requests.get(url = runQueryURL, params = searchDomain) 
  
# extracting data in json format 
searchData = rSearch.json() 
executionID = searchData['execution_id']
searchID = {'execution_id':executionID}

# Utilize executionID to retrieve results
rResults = requests.get(url = getResultsURL, params = searchID)
resultsData = rResults.json()
print(resultsData)
downloadURL = resultsData['results']

# Wait for file to be created. - 30 seconds seems to be long enough for most queries
time.sleep(30)

# Load output into dataframe - re-run the following section of code if the initial process fails.
s=requests.get(downloadURL).content
dfhosts=pd.read_csv(io.StringIO(s.decode('utf-8')))
dfhosts

In [ ]:
# Load an external notebook with normalized functions
%run ./centralprocessing.ipynb
# Pass the central function the dataframe and the column containing the IP address
df_min = get_location(dfhosts, 'value')
df_min.head(10)

In [ ]:
# Load an external notebook with normalized functions
%run ./centralprocessing.ipynb
df_plot = prepare_location(df_min)
df_plot.head(50)

In [ ]:
# Load an external notebook with normalized functions
%run ./centralprocessing.ipynb
fig = get_heatmap(df_plot)
fig